In [47]:
# Common imports
import argparse
import time
import numpy as np
import os
from datetime import datetime
import sys; sys.path.append('../deepdown/')

# Import torch
from torch.utils.data import Dataset
from deepdown.utils.data_generators import DataGenerator
import torch

# Utils
from utils.data_loader import load_target_data, load_input_data
from utils.loss_fcts import *
from utils.data_generators import DataGenerator
from utils.helpers import print_cuda_availability, DEVICE
from models.srgan import Generator, Discriminator
from config import Config

### Read the configuration file and load the data

In [48]:
from argparse import Namespace
cli_args = Namespace(config_file="../config.yaml")
conf = Config(cli_args)
conf.print()

🌳 Configuration Tree 
┣━━ path_dem: /storage/workspaces/giub_hydro/hydro/data/SRTM/srtm_1k.tif
┣━━ path_era5land: /storage/workspaces/giub_hydro/hydro/data/ERA5_Land/Switzerland-daily/
┣━━ path_mch: /storage/workspaces/giub_hydro/hydro/data/MeteoSwiss/
┣━━ path_tmp: /storage/workspaces/giub_hydro/hydro/data/tmp/pickle/files
┣━━ target_vars: ['tp', 't', 't_max', 't_min']
┣━━ output_vars: ['tp', 't']
┣━━ input_vars: {'band_data': None, 'tp': None, 't2m': None, 't2m_min': None, 't2m_max': None}
┣━━ do_crop: True
┣━━ levels: [850, 1000]
┣━━ resol_low: 0.1
┣━━ lon_limits: [2700000, 2760000]
┣━━ lat_limits: [1190000, 1260000]
┣━━ date_start: 1979-01-01
┣━━ date_end: 2021-12-31
┣━━ years_train: [1979, 2015]
┣━━ years_valid: [2015, 2018]
┣━━ years_test: [2019, 2021]
┣━━ learning_rate: 0.0002
┣━━ batch_size: 32
┣━━ num_epochs: 10
┗━━ config_file: ../config.yaml

In [49]:
# Date parameters
date_start = conf.config.date_start
date_end = conf.config.date_end

# Years for training, validation, and testing
years_train = conf.config.years_train
years_valid = conf.config.years_valid
years_test = conf.config.years_test

# Other parameters
levels = conf.config.levels
resol_low = conf.config.resol_low

# Input variables and paths
input_vars = conf.config.input_vars
input_paths = [
    conf.config.path_era5land + '/precipitation',
    conf.config.path_era5land + '/temperature',
    conf.config.path_era5land + '/max_temperature/',
    conf.config.path_era5land + '/min_temperature/'
]

# Crop parameters
do_crop = conf.config.do_crop
crop_x = conf.config.lon_limits
crop_y = conf.config.lat_limits

output_vars = conf.config.output_vars
target_paths = [
    conf.config.path_mch + '/RhiresD_v2.0_swiss.lv95/',
    conf.config.path_mch + '/TabsD_v2.0_swiss.lv95/',
    conf.config.path_mch + '/TmaxD_v2.0_swiss.lv95/',
    conf.config.path_mch + '/TminD_v2.0_swiss.lv95/'
]

# Hyperparameters
lr = conf.get('lr', 0.0002)
batch_size = conf.get('batch_size', 32)
num_epochs = conf.get('num_epochs', 100)


In [50]:
# Load target data
target = load_target_data(date_start, date_end, target_paths,
                              path_tmp=conf.config.path_tmp)



Target data loaded from pickle.


In [51]:
# Extract the axes of the final target domain based on temperature 
x_axis = target.x
y_axis = target.y

In [52]:
input_data = load_input_data(date_start=date_start, date_end=date_end, levels = levels, resol_low = 0.1, x_axis = x_axis, y_axis= y_axis, 
                                 paths = input_paths, path_dem=conf.config.path_dem, dump_data_to_pickle=True, path_tmp='../tmp/')



Extracting input data...
Extracting data for the period 1979-01-01 - 2021-12-31


Extracting data for the period 1979-01-01 - 2021-12-31
Extracting data for the period 1979-01-01 - 2021-12-31
Extracting data for the period 1979-01-01 - 2021-12-31


In [53]:
# Split the data
x_train = input_data.sel(time=slice( datetime(years_train[0],1,1), datetime(years_train[1], 12, 31)))
x_valid = input_data.sel(time=slice( datetime(years_valid[0],1,1), datetime(years_valid[1], 12, 31)))
x_test = input_data.sel(time=slice( datetime(years_test[0],1,1), datetime(years_test[1], 12, 31)))

y_train = target.sel(time=slice( datetime(years_train[0],1,1), datetime(years_train[1], 12, 31)))
y_valid = target.sel(time=slice( datetime(years_valid[0],1,1), datetime(years_valid[1], 12, 31)))
y_test = target.sel(time=slice( datetime(years_test[0],1,1), datetime(years_test[1], 12, 31)))



In [54]:
training_set = DataGenerator(inputs=x_train, outputs=y_train, input_vars=input_vars, 
              output_vars=output_vars, do_crop= True, crop_x = crop_x, crop_y=crop_y, shuffle=True, load=False,
                 mean=None, std=None, y_mean=None, y_std=None, tp_log=None)
loader_train = torch.utils.data.DataLoader(training_set, batch_size=32)
valid_set = DataGenerator(x_valid, y_valid, input_vars, output_vars, do_crop= True, crop_x = crop_x, crop_y=crop_y, shuffle=False, load=False,
                              mean=training_set.mean, std=training_set.std)
loader_val = torch.utils.data.DataLoader(valid_set, batch_size=32)
test_set = DataGenerator(x_test, y_test, input_vars, output_vars, do_crop= True, crop_x = crop_x, crop_y=crop_y, shuffle=False, load=False,
                             mean=training_set.mean, std=training_set.std)
loader_test = torch.utils.data.DataLoader(test_set, batch_size=32)

Computing/assigning mean and std...


HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 476 in H5O__attr_open_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #006: H5Adense.c line 394 in H5A__dense_open(): can't locate attribute in name index
    major: Attribute
    minor: Object not 

number of samples 13514
Computing/assigning mean and std...
number of samples 1461
Computing/assigning mean and std...
number of samples 1096
